<a href="https://colab.research.google.com/github/mingd00/News-RAG/blob/main/news_api2json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import json
import re
from urllib.parse import urlparse
from email.utils import parsedate_to_datetime

In [ ]:
def get_article_content(url: str) -> str:
    """
    주어진 URL에서 기사 본문을 추출하는 함수.

    :param url: 뉴스 기사 페이지의 URL
    :return: 기사 본문 텍스트 (사진은 제외됨)
    """
    try:
        # 웹 페이지 요청 보내기
        response = requests.get(url)

        # 요청이 성공적일 때
        if response.status_code == 200:
            # BeautifulSoup을 사용해 HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')

            # 기사 본문이 들어있는 <article> 태그 찾기
            article = soup.find('article', {'id': 'dic_area', 'class': 'go_trans _article_content'})

            # 기사 본문 추출
            if article:
                content = article.get_text(strip=True)
                return content
            else:
                return "기사 본문을 찾을 수 없습니다."
        else:
            return f"웹사이트 요청 실패: {response.status_code}"

    except Exception as e:
        return f"오류 발생: {str(e)}"


# 사용 예시
url = 'https://n.news.naver.com/mnews/article/421/0008233058?sid=102'  # 여기에 실제 뉴스 기사 URL을 넣으세요
article_content = get_article_content(url)
print(article_content)

최경주가 2024년 5월15일 제주 서귀포 핀크스 콜프클럽에서 열린 한국프로골프투어(KPGA) SK텔레콤 채리티 오픈(자선경기)에서 10번홀 세컨 아이언샷을 하고 있다. (SK텔레콤 제공) 2024.5.15/뉴스1(제주=뉴스1) 고동명 기자 = 5월 한 달간 제주도내에서 총 22개 스포츠대회가 개최된다고 6일 제주도가 밝혔다.도에 따르면 국제대회로는 오는 24~25일 탑동공원과 성판악 일대에서 '제24회 제주 국제 울트라마라톤 대회'가 열린다. 이 대회에선 10개국 선수와 관계자 등 200여 명이 참가한다.전국대회로는 8일 'KPGA 클래식 골프대회'와 15일 'SK텔레콤2025 골프대회'가 연달아 개최된다. 전국에서 모인 프로골프선수 240여 명이 이들 대회에서 최강자를 가린다. 15일에는 '제80회 '제주 전국 학생테니스 선수권대회'도 예정돼 있다.도내 대회는 지난 3일 열린 제3회 '제주특별자치도체육회장배 전도파크골프대회'를 시작으로 4일 '2025 도족구협회장배 전도족구대회', 10일 '2025 제37회 미스터제주 챔피언십', 17일 '제17회 미디어제주기 전도 유소년 축구대회' 등이 도내 곳곳에서 펼쳐진다.이외에도 △2025 제주 챌린저컵 유소년 축구 리그 대회(3~4일, 외도구장) △제26회 KCTV배 제주도 배드민턴 대회(10~11일, 제주복합체육관) △제25회 도협회장배 전도여성게이트볼대회(24일, 제주복합체육관) 등 다양한 종목 대회가 열린다.장애인대회로는 9일 '제19회 제주도지사배 전국장애인사이클대회'를 시작으로 11일 '제3회 제주도장애인 수영연맹회장배 어울림수영대회', 15일 '제16회 전도시각장애인플라잉디스크대회 및 한·일 국제플라잉디스크교류대회', 31일 '제4회 제주시 어울림 생활체육대회'가 열린다.도는 이달 열리는 국제대회에선 11억 2600만 원, 전국대회는 80억 8000만 원, 도내대회는 3억 8200만 원 등의 지역경제 파급효과를 예상하고 있다.


In [ ]:
news_media_mapping = {
    "yonhapnews.co.kr": "연합뉴스",
    "yonhapnewstv.co.kr": "연합뉴스TV",
    "news1.kr": "뉴스1",
    "edu.donga.com": "동아일보",
    "biz.heraldcorp.com": "헤럴드경제",
    "daily.hankooki.com": "한국일보",
    "kmib.co.kr": "기독교일보",
    "kbs.co.kr": "KBS",
    "munhwa.com": "문화일보",
    "sports.naver.com": "네이버 스포츠",
    "newsis.com": "뉴시스",
    "segye.com": "세계일보",
    "hankooki.com": "한겨레",
    "chosun.com": "조선일보",
    "joongang.co.kr": "중앙일보",
    "khan.co.kr": "경향신문",
    "hani.co.kr": "한겨레",
    "sports.chosun.com": "스포츠조선",
    "sports.donga.com": "스포츠동아",
    "news.naver.com": "네이버 뉴스",
    "n.news.naver.com": "네이버 뉴스",
    "m.post.naver.com": "네이버 포스트",
    "seoul.co.kr": "서울신문",
    "etnews.com": "전자신문",
    "ytn.co.kr": "YTN",
    "tbs.seoul.kr": "TBS",
    "mbn.co.kr": "MBN 뉴스",
    "news.kmib.co.kr": "국민일보",
    "newdaily.co.kr": "뉴데일리",
    "yna.co.kr": "연합뉴스",
    "naeil.com": "내일신문",
    "kihoilbo.co.kr": "기호일보",
    "edaily.co.kr": "이데일리",
    "fnnews.com": "파이넨셜뉴스",
    "hankyung.com": "한경",
    "incheonnews.com": "인천뉴스",
    "bloter.net": "BROTER",
    "dt.co.kr": "디지털타임스",
    "sentv.co.kr": "서울경제TV" ,
    "econovill.com": "이코노믹 리뷰",
    "nytimes.com": "The New York Times",
    "digitaltoday.co.kr": "Digital Today",
    "koreajoongangdaily.joins.com": "Korea JoongAng Daily",
    "ddaily.co.kr": "디지털 데일리",
    "hankyung.com": "한국경제신문",
    "sedaily.com": "서울경제",
    "sportsseoul.com": "스포츠서울",
    "joongboo.com": "중부일보",
    "nocutnews.co.kr": "노컷뉴스",
    "imbc.com": "MBC 뉴스",
    "news.mt.co.kr": "머니투데이",
}

In [ ]:
# URL에서 도메인 추출 함수
def extract_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.hostname
    if domain.startswith("www."):
        domain = domain[4:]  # "www." 제거
    return domain

In [ ]:
# 태그 제거
def clean_html_tags(text):
    return re.sub(r"<.*?>", "", text)

# 날짜 형식 지정
def format_date(pub_date_str):
    try:
        dt = parsedate_to_datetime(pub_date_str)
        return dt.strftime("%Y-%m-%d %H:%M")  # 예: 2025-05-06 10:59
    except Exception as e:
        print(f"날짜 파싱 오류: {e}")
        return pub_date_str

In [ ]:
# 네이버 API 키 (자신의 API 키로 교체하세요)
client_id = "gh33NVdUbiTCVrbFAQhV"
client_secret = "Wg6SoJlOj6"

# 뉴스 데이터를 담을 리스트
news_data = []

In [ ]:
# 요청 URL을 위한 기본 설정
base_url = "https://openapi.naver.com/v1/search/news.json"

# 네이버 API 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

In [ ]:
keywords = [
    "국내 정치", "외교", "선거", "정치",
    "금융", "부동산", "기업", "주식", "경제",
    "사건", "교육", "환경", "사고", "사회",
    "공연", "문학", "트렌드", "문화",
    "국제 정치", "해외 사건", "세계 경제", "국제",
    "축구", "야구", "골프", "스포츠",
    "드라마", "가요", "연예계 이슈", "영화", "연예",
    "인공지능", "스마트폰", "우주", "IT", "LLM", "AI", "과학",
    "날씨", "건강", "여행", "요리", "생활",
    "제조업", "에너지", "유통", "산업",
    "노동", "기후 변화", "반도체"
]

In [ ]:
total_news_count = 1000
display_count = 100
total_pages = total_news_count // display_count

news_data = []

for keyword in keywords:
    print(f"🔍 '{keyword}' 키워드 수집 시작, 현재까지 리스트 길이: {len(news_data)}")

    for start in range(1, total_pages + 1):
        params = {
            "query": keyword,
            "display": display_count,
            "start": (start - 1) * display_count + 1,
            "sort": "sim"
        }

        try:
            response = requests.get(base_url, headers=headers, params=params)
            data = response.json()

            for item in data["items"]:
                if item["link"].startswith("https://n.news.naver.com/"):
                    news_item = {
                        "title": clean_html_tags(item["title"]),
                        "date": format_date(item["pubDate"]),
                        "link": item["link"],
                        "content": get_article_content(item["link"]),
                        "journal": news_media_mapping.get(extract_domain(item["originallink"]), "Unknown")
                    }
                    news_data.append(news_item)

            time.sleep(1)

        except Exception as e:
            print(f"❌ '{keyword}' 키워드의 {start}페이지에서 오류 발생: {e}")
            break

print(f"✅ 전체 수집 완료! 총 수집된 기사 수: {len(news_data)}")


🔍 '국내 정치' 키워드 수집 시작, 현재까지 리스트 길이: 0
❌ '국내 정치' 키워드의 5페이지에서 오류 발생: 'NoneType' object has no attribute 'startswith'
🔍 '외교' 키워드 수집 시작, 현재까지 리스트 길이: 312
🔍 '선거' 키워드 수집 시작, 현재까지 리스트 길이: 1081
🔍 '정치' 키워드 수집 시작, 현재까지 리스트 길이: 1943
🔍 '금융' 키워드 수집 시작, 현재까지 리스트 길이: 2797
🔍 '부동산' 키워드 수집 시작, 현재까지 리스트 길이: 3288
🔍 '기업' 키워드 수집 시작, 현재까지 리스트 길이: 3914
🔍 '주식' 키워드 수집 시작, 현재까지 리스트 길이: 4453
🔍 '경제' 키워드 수집 시작, 현재까지 리스트 길이: 4982
🔍 '사건' 키워드 수집 시작, 현재까지 리스트 길이: 5717
🔍 '교육' 키워드 수집 시작, 현재까지 리스트 길이: 6590
🔍 '환경' 키워드 수집 시작, 현재까지 리스트 길이: 6994
🔍 '사고' 키워드 수집 시작, 현재까지 리스트 길이: 7472
🔍 '사회' 키워드 수집 시작, 현재까지 리스트 길이: 8186
🔍 '공연' 키워드 수집 시작, 현재까지 리스트 길이: 8713
🔍 '문학' 키워드 수집 시작, 현재까지 리스트 길이: 9140
🔍 '트렌드' 키워드 수집 시작, 현재까지 리스트 길이: 9583
🔍 '문화' 키워드 수집 시작, 현재까지 리스트 길이: 9901
🔍 '국제 정치' 키워드 수집 시작, 현재까지 리스트 길이: 10287
🔍 '해외 사건' 키워드 수집 시작, 현재까지 리스트 길이: 10834
🔍 '세계 경제' 키워드 수집 시작, 현재까지 리스트 길이: 11441
🔍 '국제' 키워드 수집 시작, 현재까지 리스트 길이: 12060
🔍 '축구' 키워드 수집 시작, 현재까지 리스트 길이: 12516
❌ '축구' 키워드의 9페이지에서 오류 발생: 'NoneType' object has no attribute 'startswith'
🔍 '야구'

In [ ]:
# 결과를 JSON 파일로 저장
with open("sports_news.json", "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"총 {len(news_data)}개의 뉴스가 저장되었습니다.")

총 23315개의 뉴스가 저장되었습니다.


In [ ]:
# 저장된 JSON 파일 열기
with open("sports_news.json", "r", encoding="utf-8") as f:
    news_data = json.load(f)

# 첫 번째 뉴스의 content 확인
first_news_content = news_data[0]

print(first_news_content)

{'title': '체코 원전 계약 제동…국내 건설업계 ‘초긴장’', 'date': '2025-05-07 10:20', 'link': 'https://n.news.naver.com/mnews/article/003/0013225746?sid=101', 'content': '두산에너빌리티·대우건설 수조원 수주 여부 달려[두코바니(체코)=AP/뉴시스]체코 두코바니에 있는 두코바니 원자력발전소의 냉각탑 4개의 모습.  2022.11.30[서울=뉴시스] 김종민 기자 = 체코 법원이 한국수력원자력(한수원)의 신규 원전 건설 계약에 제동을 걸면서, 국내 원전 관련 건설·기자재 업체들이 비상에 걸렸다. 특히 두산에너빌리티와 대우건설 등 핵심 기업들의 수조 원대 수주 여부가 불투명해지며 건설업계 전반에 긴장감이 감돌고 있다.체코 브르노 지방 법원은 프랑스전력공사(EDF)가 제기한 가처분 신청을 일부 인용하며, 한수원과 체코전력공사(CEZ) 간 7일 예정돼 있던 최종 계약서 서명을 금지했다. 이에 따라 양국 정부와 국회 대표단까지 참석한 계약 서명식은 전격 취소됐으며, 한국 대표단은 별다른 성과 없이 귀국길에 오를 수밖에 없게 됐다.이번 가처분 결정은 EDF가 체코 공정거래청의 입찰 승인 과정에 문제가 있다며 소송을 제기한 데 따른 것으로, 법원은 “회복 불가능한 결과를 초래할 수 있다”는 이유로 계약 체결을 일시 중단시켰다.문제가 된 체코 원전 프로젝트는 한수원이 주관하고, 한전기술, 한전KPS, 한전원자력연료 등 한전 계열사와 함께 두산에너빌리티, 대우건설 등이 참여하는 ‘팀 코리아’ 형태로 진행돼 왔다. 전체 사업 규모는 약 26조원 이상으로 추산되며, 이 가운데 두산에너빌리티는 원자로 등 주기기 공급, 대우건설은 주요 시공을 담당하는 핵심 역할을 맡고 있다.이번 법원 결정은 단순한 일정 지연을 넘어, 유럽 내 원전 수출 전반에 불확실성을 키우는 ‘법률 리스크’의 시작이라는 우려도 커지고 있다. EDF는 유럽연합(EU) 집행위원회에도 한국 정부의 지원이 EU 보조금 규제에

In [ ]:
# "journal" 값이 "Unknown"인 데이터 개수 세기
unknown_count = sum(1 for item in news_data if item['journal'] == 'Unknown')

# 개수 출력
print(f'Unknown인 데이터 개수: {unknown_count}')

Unknown인 데이터 개수: 8257


In [ ]:
# "journal" 값이 "Unknown"이 아닌 데이터만 남김
filtered_data = [item for item in news_data if item['journal'] != 'Unknown']

# 새로운 JSON 파일로 저장
with open('filtered_news_data.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print(f'Unknown 값을 제거한 후 남은 데이터 개수: {len(filtered_data)}')

Unknown 값을 제거한 후 남은 데이터 개수: 15058


In [3]:
# 저장된 JSON 파일 열기
with open("filtered_news_data.json", "r", encoding="utf-8") as f:
    news_data = json.load(f)

# 첫 번째 뉴스의 content 확인
first_news_content = news_data[0]

print(first_news_content)

{'title': '체코 원전 계약 제동…국내 건설업계 ‘초긴장’', 'date': '2025-05-07 10:20', 'link': 'https://n.news.naver.com/mnews/article/003/0013225746?sid=101', 'content': '두산에너빌리티·대우건설 수조원 수주 여부 달려[두코바니(체코)=AP/뉴시스]체코 두코바니에 있는 두코바니 원자력발전소의 냉각탑 4개의 모습.  2022.11.30[서울=뉴시스] 김종민 기자 = 체코 법원이 한국수력원자력(한수원)의 신규 원전 건설 계약에 제동을 걸면서, 국내 원전 관련 건설·기자재 업체들이 비상에 걸렸다. 특히 두산에너빌리티와 대우건설 등 핵심 기업들의 수조 원대 수주 여부가 불투명해지며 건설업계 전반에 긴장감이 감돌고 있다.체코 브르노 지방 법원은 프랑스전력공사(EDF)가 제기한 가처분 신청을 일부 인용하며, 한수원과 체코전력공사(CEZ) 간 7일 예정돼 있던 최종 계약서 서명을 금지했다. 이에 따라 양국 정부와 국회 대표단까지 참석한 계약 서명식은 전격 취소됐으며, 한국 대표단은 별다른 성과 없이 귀국길에 오를 수밖에 없게 됐다.이번 가처분 결정은 EDF가 체코 공정거래청의 입찰 승인 과정에 문제가 있다며 소송을 제기한 데 따른 것으로, 법원은 “회복 불가능한 결과를 초래할 수 있다”는 이유로 계약 체결을 일시 중단시켰다.문제가 된 체코 원전 프로젝트는 한수원이 주관하고, 한전기술, 한전KPS, 한전원자력연료 등 한전 계열사와 함께 두산에너빌리티, 대우건설 등이 참여하는 ‘팀 코리아’ 형태로 진행돼 왔다. 전체 사업 규모는 약 26조원 이상으로 추산되며, 이 가운데 두산에너빌리티는 원자로 등 주기기 공급, 대우건설은 주요 시공을 담당하는 핵심 역할을 맡고 있다.이번 법원 결정은 단순한 일정 지연을 넘어, 유럽 내 원전 수출 전반에 불확실성을 키우는 ‘법률 리스크’의 시작이라는 우려도 커지고 있다. EDF는 유럽연합(EU) 집행위원회에도 한국 정부의 지원이 EU 보조금 규제에

In [6]:
# 조건에 맞는 항목 개수 세기
count = sum(1 for item in news_data if item.get("content") == "본문을 크롤링할 수 없습니다.")

print(f'"본문을 크롤링할 수 없습니다."인 항목 수: {count}')

"본문을 크롤링할 수 없습니다."인 항목 수: 0
